In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import json
import sys
from pathlib import Path
import re 

import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
from IPython.display import display
from ipydatagrid import DataGrid

root_path = Path('/dev/projects/cds/')

if (path := str(root_path.resolve())) not in sys.path:
    sys.path.append(path)

from api.blp import BlpQuery
from api.data.base import data_path
from api.data.curves import curves
from api.pricing.cds import get_isda_cds, get_isda_cds_specs
from api.pricing.utils import (
    get_cds_specs,
    get_cds_curve,
    get_cds_spreads,
    get_risk_free_curve,
    get_cds_implied_price
)

In [4]:
# Technically, the model requires the discount factor to be 1 at spot;
# but we can't do that and also have the discount curve extend back to
# the trade date.  For the time being, we'll keep discount = 1 at trade.
# The results match anyway.

#### Settings

In [5]:
cds_name = "TURKEY CDS USD SR 5Y D14 Curncy"
curve_name =  "USD ISDA Standard Rates Swap Curve" # "EUR ISDA Standard Rates Swap Curve"  # 'USD ISDA CDS SOFR Rates Swap Curve'
trade_date = pd.Timestamp('2021-06-02') # pd.Timestamp.today().floor('d')

date_start = pd.Timestamp('2021-06-02')
date_end = pd.Timestamp('2022-06-23')

dt_fmt = '%Y-%m-%d'
bbg_dt_fmt = '%Y%m%d'

#### Bloomberg Connection

In [6]:
bq = BlpQuery(timeout=50000).start()

#### Local data

In [7]:
curve_hist = pd.read_csv(data_path / 'curve_history.csv', parse_dates=['date'], dayfirst=True)
cds_ref = pd.read_csv(data_path / 'cds_reference.csv', parse_dates=['maturity'], dayfirst=True)
cds_hist = pd.read_csv(data_path / 'cds_history.csv', parse_dates=['date'], dayfirst=True)

In [9]:
cds_specs = get_cds_specs(cds_name, cds_ref)
cds_curve = get_cds_curve(cds_name, cds_ref, cds_hist)
cds_spread  = get_cds_spreads(cds_name, cds_hist).loc[date_start:date_end]
risk_free_curve = get_risk_free_curve(curve_name, curves, curve_hist)
notional = 10000000
coupon = 100
flat = True
n = 10

In [10]:
cds_ip = get_cds_implied_price(
    cds_name=cds_name,
    dates=cds_spread.index[:n],
    spreads=cds_spread['last_price'][:n]
)

In [12]:
res = {}
for i, (date, row) in enumerate(cds_spread.iterrows()):
    rfc = risk_free_curve.query(f'date == "{date:{dt_fmt}}"').set_index('tenor')['last_price']
    cds_curve_ = cds_curve.loc[cds_curve['date'] == date].set_index('tenor')['last_price']
    spread = row['last_price']
    
    cds = get_isda_cds_specs(
        cds_specs=cds_specs,
        risk_free_curve=rfc,
        trade_date=date,
        cds_spread=spread,
        notional=notional,
        coupon=coupon
    )
    res[date] = {'npv': cds.NPV(), 'cds_implied_price': 100 * (1+cds.fairUpfront()), 'spread': spread}
    
    if n and i + 1 == n:
        break

    # print(f"{date:%Y-%m-%d} NPV: {cds.NPV()}, Price: {100 * (1+cds.fairUpfront())}, spread: {spread}")
    # break

In [13]:
res = pd.DataFrame(res).T
res

npv  cds_implied_price  spread
2021-06-02  1.346058e+06          86.539251   405.0
2021-06-03  1.365644e+06          86.343398   410.0
2021-06-04  1.342374e+06          86.576111   404.5
2021-06-07  1.321645e+06          86.783462   399.5
2021-06-08  1.292981e+06          87.070104   392.5
2021-06-09  1.252680e+06          87.473065   382.5
2021-06-10  1.212229e+06          87.877580   372.5
2021-06-11  1.206049e+06          87.939390   371.0
2021-06-14  1.221914e+06          87.780781   375.5
2021-06-15  1.254854e+06          87.451385   384.0

In [ ]:
pe = res['cds_implied_price'].rename('gio_pricer')
be = cds_ip.set_index('date')['cds_implied_price'].rename('bbg_pricer')
diff = (pe - be).rename('diff')

In [ ]:
pd.concat([pe, be, diff], axis=1)